## Plan to focus on central intermediary metabolism [BC-1.7] {6 members} 
- [x] EG11685 - Isoprimeverose
- [x] EG10456 - Nitrous Oxide
- [x] G6286 - OXAMATE
- [x] EG11017 - TREHALOSE
- [x] EG10033 - Glucose-1-phosphate
- [x] EG10369 - GLC-D-LACTONE

In [1]:
import numpy as np
import ast
import pandas as pd
import os
import matplotlib.pyplot as plt
import dill
import requests
import xmltodict
import cvxpy as cp
import itertools
import networkx as nx
import plotly.express as px

from scipy.special import logsumexp

%matplotlib inline
# sns.set(style='darkgrid', palette='viridis', context='talk')

os.chdir(os.path.expanduser('~/dev/vivarium-ecoli'))

from ecoli.processes.metabolism_redux_classic import NetworkFlowModel, FlowResult
from ecoli.library.schema import numpy_schema, bulk_name_to_idx, listener_schema, counts
from ecoli.processes.registries import topology_registry
TOPOLOGY = topology_registry.access("ecoli-metabolism-redux")

In [2]:
# load basal new model
time = '400'
date = '2025-01-30'
experiment = 'NEW_updated_bad_rxn'
condition = 'basal'
entry = f'{experiment}_{time}_{date}'
folder = f'out/metabolism-comparison/{condition}/{entry}/'

output = np.load(folder + '0_output.npy',allow_pickle='TRUE').item()
# output = np.load(r"out/geneRxnVerifData/output_glc.npy", allow_pickle=True, encoding='ASCII').tolist()
output = output['agents']['0']
fba_basal = output['listeners']['fba_results']
bulk = pd.DataFrame(output['bulk'])
f = open(folder + 'agent_steps.pkl', 'rb')
agent_basal = dill.load(f)
f.close()

In [3]:
# get commonly stored variables
metabolism_basal = agent_basal['ecoli-metabolism-redux-classic']
stoichiometry = metabolism_basal.stoichiometry.copy()
reaction_names = metabolism_basal.reaction_names
kinetic_reaction_ids = metabolism_basal.kinetic_constraint_reactions
fba_new_reaction_ids = metabolism_basal.parameters["fba_new_reaction_ids"]
fba_reaction_ids_to_base_reaction_ids = metabolism_basal.parameters['fba_reaction_ids_to_base_reaction_ids']
binary_kinetic_idx = metabolism_basal.binary_kinetic_idx
metabolites = metabolism_basal.metabolite_names

S = stoichiometry.copy()
S = pd.DataFrame(S, index=metabolites, columns=reaction_names)
homeostatic_count = pd.DataFrame(fba_basal["homeostatic_metabolite_counts"], columns=metabolism_basal.homeostatic_metabolites).loc[24, :]
homeostatic = pd.DataFrame(fba_basal["target_homeostatic_dmdt"], columns=metabolism_basal.homeostatic_metabolites).loc[24, :]
maintenance = pd.DataFrame(fba_basal["maintenance_target"][1:], columns=['maintenance_reaction']).iat[24, 0]

FREE_RXNS = ["TRANS-RXN-145", "TRANS-RXN0-545", "TRANS-RXN0-474"]

## load variables from track_reaction_usage.ipynb

In [4]:
%store -r df_reaction_flux df_reaction_track df_gene_usage unused_kinetic_gene

In [5]:
genes_of_interest = ['EG11685','EG10456','G6286','EG11017','EG10033','EG10369']
unused_kinetic_gene.loc[genes_of_interest]

# not all reactions in the unused_kinetic_genes are included in the model
# here, I am manually extracting rxns of the genes of interest
rxns_of_interest = ['RXN-12402', 'RXN-12402-CPD0-1202/WATER//ALPHA-GLUCOSE/CPD-25028.41.', 'RXN-12402-CPD0-1202/WATER//GLC/CPD-25028.31.',
                    'R621-RXN-NITRIC-OXIDE/NADH/OXYGEN-MOLECULE//NITRATE/NAD/PROTON.54.',
                    'R621-RXN-NITRIC-OXIDE/NADPH/OXYGEN-MOLECULE//NITRATE/NADP/PROTON.56.',
                    'RXN0-7024', 'TREHALA-RXN[CCO-PERI-BAC]-TREHALOSE/WATER//GLC/ALPHA-GLUCOSE.49.',
                    'GLUCOSE-1-PHOSPHAT-RXN[CCO-PERI-BAC]-GLC-1-P/WATER//ALPHA-GLUCOSE/Pi.46.',
                    'GLUCOSE-1-PHOSPHAT-RXN[CCO-PERI-BAC]-GLC-1-P/WATER//GLC/Pi.36.',
                    'GLUCOSE-1-PHOSPHAT-RXN[CCO-PERI-BAC]-GLC-1-P/WATER//Glucopyranose/Pi.46.', 'RXN0-1001',
                    'RXN0-6373-ALPHA-GLUCOSE/UBIQUINONE-8//GLC-D-LACTONE/CPD-9956.51.',
                    'RXN0-6373-GLC/UBIQUINONE-8//GLC-D-LACTONE/CPD-9956.41.',
                    'RXN0-6373-Glucopyranose/UBIQUINONE-8//GLC-D-LACTONE/CPD-9956.51.',
                    ]
df_reaction_flux.loc[rxns_of_interest]

,basal,acetate,rich,anaerobic_basal,anaerobic_acetate,anaerobic_rich,is_new,is_kinetic
RXN-12402,0.0,0.0,0.0,-0.0,-0.0,False,True,True
RXN-12402-CPD0-1202/WATER//ALPHA-GLUCOSE/CPD-25028.41.,0.0,0.0,0.0,-0.0,-0.0,False,True,False
RXN-12402-CPD0-1202/WATER//GLC/CPD-25028.31.,0.0,0.0,0.0,-0.0,-0.0,False,True,False
R621-RXN-NITRIC-OXIDE/NADH/OXYGEN-MOLECULE//NITRATE/NAD/PROTON.54.,0.0,0.0,0.0,-0.0,-0.0,False,True,True
R621-RXN-NITRIC-OXIDE/NADPH/OXYGEN-MOLECULE//NITRATE/NADP/PROTON.56.,0.0,0.0,0.0,-0.0,-0.0,False,True,False
RXN0-7024,0.0,0.0,0.0,-0.0,-0.0,False,True,True
TREHALA-RXN[CCO-PERI-BAC]-TREHALOSE/WATER//GLC/ALPHA-GLUCOSE.49.,0.0,0.0,0.0,0.0,0.0,False,True,True
GLUCOSE-1-PHOSPHAT-RXN[CCO-PERI-BAC]-GLC-1-P/WATER//ALPHA-GLUCOSE/Pi.46.,0.0,0.0,0.0,-0.0,-0.0,False,True,False
GLUCOSE-1-PHOSPHAT-RXN[CCO-PERI-BAC]-GLC-1-P/WATER//GLC/Pi.36.,0.0,0.0,0.0,-0.0,-0.0,False,True,False
GLUCOSE-1-PHOSPHAT-RXN[CCO-PERI-BAC]-GLC-1-P/WATER//Glucopyranose/Pi.46.,0.0,0.0,0.0,-0.0,-0.0,False,True,False


## Define functions for testing

In [6]:
def get_subset_S(S, met_of_interest):
    S_met = S.loc[met_of_interest, :]
    S_met = S_met.loc[:,~np.all(S_met == 0, axis=0)]
    return S_met, S_met.columns

In [7]:
def test_NetworkFlowModel(objective_weights,
                          uptake_addition = set([]), uptake_removal = set([]), new_exchange_molecules = set([]),
                          add_metabolite = None, add_reaction = None, add_kinetic = None, remove_reaction = None, force_reaction = None):
    # update exchanges
    uptake = metabolism_basal.allowed_exchange_uptake.copy()
    uptake = set(uptake)
    uptake = uptake | uptake_addition
    uptake = uptake - uptake_removal
    
    exchange_molecules = metabolism_basal.exchange_molecules.copy()
    exchange_molecules = exchange_molecules | new_exchange_molecules
    
    # update stoichiometry
    reaction_names = metabolism_basal.reaction_names.copy()
    kinetic_reaction_ids = metabolism_basal.kinetic_constraint_reactions.copy()
    kinetic = pd.DataFrame(fba_basal["target_kinetic_fluxes"], columns=metabolism_basal.kinetic_constraint_reactions).loc[24, :].copy()
    metabolites = metabolism_basal.metabolite_names.copy()
    
    S_new = stoichiometry.copy()
    
    if add_metabolite is not None: # add to metabolites list because they are currently not included in the model
        for m in add_metabolite:
            if m not in metabolites:
                metabolites.append(m)
        # append rows of zeros to S_new of length add_metabolite
        S_new = np.concatenate((S_new, np.zeros((len(add_metabolite), S_new.shape[1]))), axis=0)
        print(S_new.shape)
    
    if add_reaction is not None:
        # assert add_reaction is a dictionary
        assert isinstance(add_reaction, dict)
        
        for r,s in add_reaction.items():
            if r not in reaction_names:
                reaction_names.append(r)
            # append columns of reaction stoich to S_new of length add_reaction
            new_reaction = np.zeros((S_new.shape[0], 1))
            for m, v in s.items():
                new_reaction[metabolites.index(m), 0] = v
            S_new = np.concatenate((S_new, new_reaction), axis=1)
        print(S_new.shape)
    
    if add_kinetic is not None:
        # assert add_kinetic is a dictionary
        assert isinstance(add_kinetic, dict)
        
        for r, v in add_kinetic.items():
            if r not in kinetic_reaction_ids:
                kinetic_reaction_ids.append(r)
                kinetic[r] = v
                
    if remove_reaction is not None:
        for r in remove_reaction:
            r_idx = reaction_names.index(r)
            S_new = np.delete(S_new, r_idx, axis=1)
            reaction_names.remove(r)
            if r in kinetic_reaction_ids:
                kinetic_reaction_ids.remove(r)
                del kinetic[r]
                
    if force_reaction is not None:
        force_reaction_idx = np.array([reaction_names.index(r) for r in force_reaction])
    else:
        force_reaction_idx = force_reaction

    print(f'Force Reaction ID is:',force_reaction_idx)
    # Solve NetworkFlowModel
    model = NetworkFlowModel(
            stoich_arr=S_new,
            metabolites=metabolites,
            reactions=reaction_names,
            homeostatic_metabolites=metabolism_basal.homeostatic_metabolites,
            kinetic_reactions=kinetic_reaction_ids,
            free_reactions=FREE_RXNS)
    model.set_up_exchanges(exchanges=exchange_molecules, uptakes=uptake)
    solution: FlowResult = model.solve(
            homeostatic_concs=homeostatic_count * metabolism_basal.counts_to_molar.asNumber(), # in conc
            homeostatic_dm_targets=np.array(list(dict(homeostatic).values())), # *10^7
            maintenance_target=maintenance, # *10^6 ish
            kinetic_targets=np.array(list(dict(kinetic).values())), # *10^6 ish
            # binary_kinetic_idx=binary_kinetic_idx, #7646
            binary_kinetic_idx=None,
            force_flow_idx=force_reaction_idx,
            objective_weights=objective_weights, #same
            upper_flux_bound= 1000000000, # increase to 10^9 because notebook runs FlowResult using Counts, WC runs using conc.   
            solver=cp.GLOP) #SCS. ECOS
    print(f'Optimal objective function reached is: {solution.objective}')
    return solution.velocities, reaction_names, S_new, metabolites, kinetic

## Focus on gene EG11685 --> RXN-12402s first

In [8]:
uptake_addition = set([
                       'CPD0-1202[e]',
                       ])
uptake_removal = set([
                      # "OXYGEN-MOLECULE[p]",
                      'GLC [p]'
                     ])
new_exchange_molecules = set(['CPD0-1202[e]'])
add_reaction = {
    'TEMP-TRANSPORT-CPD0-1202[p]->[c]': {
        'CPD0-1202[p]': -1,
        'CPD0-1202[c]': 1
    },
    'TEMP-TRANSPORT-CPD0-1202[p]->[c] (reverse)': {
        'CPD0-1202[p]': 1,
        'CPD0-1202[c]': -1
    }
}
# force_reaction = ['TEMP-TRANSPORT-CPD0-1202[p]->[c]']
objective_weights = {'secretion': 0.01, 'efficiency': 0.000001, 'kinetics': 0.001}


In [13]:
solution_flux, test_reaction_names, S_new, test_metabolites, test_kinetic = test_NetworkFlowModel(
                                            objective_weights, 
                                            uptake_addition=uptake_addition, uptake_removal=uptake_removal,
                                            new_exchange_molecules=new_exchange_molecules, add_reaction = add_reaction)

(6076, 9360)
Force Reaction ID is: None
Optimal objective function reached is: 39543.40176867694


In [15]:
sim_flux = pd.DataFrame({
    'flux': solution_flux,
    'is_new': [
        'New Reactions' if id in fba_new_reaction_ids 
        else 'TEMP' if id in add_reaction.keys() 
        else 'Old Reactions' 
            for id in test_reaction_names
    ]
}, index=test_reaction_names)

In [16]:
met_of_interest = ['CPD0-1202[e]', 'CPD0-1202[p]', 'CPD0-1202[c]']
S_new = pd.DataFrame(S_new, index=test_metabolites, columns=test_reaction_names)
S_met, rxns  = get_subset_S(S_new, met_of_interest)
S_met

,RXN-12402,RXN-12402-CPD0-1202/WATER//ALPHA-GLUCOSE/CPD-25028.41.,RXN-12402-CPD0-1202/WATER//GLC/CPD-25028.31.,RXN0-7203-CPD0-1202//CPD0-1202.21.,RXN0-7203-CPD0-1202//CPD0-1202.21. (reverse),RXN0-7209-CPD0-1202//CPD0-1202.21.,RXN0-7209-CPD0-1202//CPD0-1202.21. (reverse),TEMP-TRANSPORT-CPD0-1202[p]->[c],TEMP-TRANSPORT-CPD0-1202[p]->[c] (reverse)
CPD0-1202[e],0.0,0.0,0.0,1.0,-1.0,1.0,-1.0,0.0,0.0
CPD0-1202[p],0.0,0.0,0.0,-1.0,1.0,-1.0,1.0,-1.0,1.0
CPD0-1202[c],-1.0,-1.0,-1.0,0.0,0.0,0.0,0.0,1.0,-1.0


In [17]:
rxns_of_interest = ['TEMP-TRANSPORT-CPD0-1202[p]->[c]', 'TEMP-TRANSPORT-CPD0-1202[p]->[c] (reverse)',
                    'RXN-12402', 'RXN-12402-CPD0-1202/WATER//ALPHA-GLUCOSE/CPD-25028.41.',
                    'RXN-12402-CPD0-1202/WATER//GLC/CPD-25028.31.', 'RXN0-7203-CPD0-1202//CPD0-1202.21.',
                    'RXN0-7209-CPD0-1202//CPD0-1202.21.']
rxn_flux = sim_flux.loc[rxns]
rxn_flux['kinetic'] = [test_kinetic[r] if r in kinetic_reaction_ids else False for r in rxn_flux.index]
rxn_flux

,flux,is_new,kinetic
RXN-12402,77.0,New Reactions,77.0
RXN-12402-CPD0-1202/WATER//ALPHA-GLUCOSE/CPD-25028.41.,-0.0,New Reactions,False
RXN-12402-CPD0-1202/WATER//GLC/CPD-25028.31.,-0.0,New Reactions,False
RXN0-7203-CPD0-1202//CPD0-1202.21.,0.0,Old Reactions,False
RXN0-7203-CPD0-1202//CPD0-1202.21. (reverse),77.0,Old Reactions,False
RXN0-7209-CPD0-1202//CPD0-1202.21.,0.0,Old Reactions,False
RXN0-7209-CPD0-1202//CPD0-1202.21. (reverse),0.0,Old Reactions,False
TEMP-TRANSPORT-CPD0-1202[p]->[c],77.0,TEMP,False
TEMP-TRANSPORT-CPD0-1202[p]->[c] (reverse),-0.0,TEMP,False


## Focus on gene G6286 --> RXN0-7024 (1) next

In [23]:
uptake_addition = set([
                       "S-ALLANTOIN[e]"
                       ])
uptake_removal = set([
                      "OXYGEN-MOLECULE[p]",
                      'AMMONIUM[c]'
                     ])
new_exchange_molecules = set(['OXAMATE[e]', 'S-ALLANTOIN[e]'])

add_metabolite = ["OXAMATE[e]", "OXAMATE[p]"]
add_reaction = {
    'TEMP-TRANSPORT-OXAMATE[e]->[p]': {
        'OXAMATE[e]': -1,
        'OXAMATE[p]': 1
    },
    'TEMP-TRANSPORT-OXAMATE[e]->[p] (reverse)': {
        'OXAMATE[e]': 1,
        'OXAMATE[p]': -1
    },
    'TEMP-TRANSPORT-OXAMATE[p]->[c]': {
        'OXAMATE[p]': -1,
        'OXAMATE[c]': 1
    },
    'TEMP-TRANSPORT-OXAMATE[p]->[c] (reverse)': {
        'OXAMATE[p]': 1,
        'OXAMATE[c]': -1
    },
    'TEMP-OXAMATE-CARBAMOYLTRANSFERASE-RXN (reverse)':{
        "OXAMATE[c]": 1,
        "CARBAMOYL-P[c]": 1,
        "CPD-389[c]": -1,
        "Pi[c]": -1
    },
}

objective_weights = {'secretion': 0.01, 'efficiency': 0.000001, 'kinetics': 0.00001}

In [24]:
solution_flux, test_reaction_names, S_new, test_metabolites, test_kinetic = test_NetworkFlowModel(
                                            objective_weights, 
                                            uptake_addition=uptake_addition, uptake_removal=uptake_removal,
                                            new_exchange_molecules=new_exchange_molecules, add_metabolite=add_metabolite, add_reaction=add_reaction)

(6078, 9358)
(6078, 9363)
Force Reaction ID is: None
Optimal objective function reached is: 65128.38443874318


In [25]:
sim_flux = pd.DataFrame({
    'flux': solution_flux,
    'is_new': [
        'New Reactions' if id in fba_new_reaction_ids 
        else 'TEMP' if id in add_reaction.keys() 
        else 'Old Reactions' 
            for id in test_reaction_names
    ]
}, index=test_reaction_names)

In [26]:
met_of_interest = ['OXAMATE[e]', 'OXAMATE[p]', 'OXAMATE[c]', 'CPD-389[c]','CPD-1091[c]']
S_new = pd.DataFrame(S_new, index=test_metabolites, columns=test_reaction_names)
S_met, rxns  = get_subset_S(S_new, met_of_interest)
S_met

,OXAMATE-CARBAMOYLTRANSFERASE-RXN,R165-RXN-CPD-1091/NADP//CPD-389/NADPH/PROTON.36.,RXN0-7024,UREIDOGLYCOLATE-LYASE-RXN,URUR-RXN,TEMP-TRANSPORT-OXAMATE[e]->[p],TEMP-TRANSPORT-OXAMATE[e]->[p] (reverse),TEMP-TRANSPORT-OXAMATE[p]->[c],TEMP-TRANSPORT-OXAMATE[p]->[c] (reverse),TEMP-OXAMATE-CARBAMOYLTRANSFERASE-RXN (reverse)
OXAMATE[e],0.0,0.0,0.0,0.0,0.0,-1.0,1.0,0.0,0.0,0.0
OXAMATE[p],0.0,0.0,0.0,0.0,0.0,1.0,-1.0,-1.0,1.0,0.0
OXAMATE[c],-1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,-1.0,1.0
CPD-389[c],1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0
CPD-1091[c],0.0,-1.0,-1.0,-1.0,1.0,0.0,0.0,0.0,0.0,0.0


In [27]:
rxns_of_interest = ['TEMP-TRANSPORT-CPD0-1202[p]->[c]', 'TEMP-TRANSPORT-CPD0-1202[p]->[c] (reverse)',
                    'RXN-12402', 'RXN-12402-CPD0-1202/WATER//ALPHA-GLUCOSE/CPD-25028.41.',
                    'RXN-12402-CPD0-1202/WATER//GLC/CPD-25028.31.', 'RXN0-7203-CPD0-1202//CPD0-1202.21.',
                    'RXN0-7209-CPD0-1202//CPD0-1202.21.']
rxns = list(rxns)
rxns.append('ALLANTOINASE-RXN')
rxn_flux = sim_flux.loc[rxns]
rxn_flux['kinetic'] = [test_kinetic[r] if r in kinetic_reaction_ids else False for r in rxn_flux.index]
rxn_flux

,flux,is_new,kinetic
OXAMATE-CARBAMOYLTRANSFERASE-RXN,-0.000000,Old Reactions,False
R165-RXN-CPD-1091/NADP//CPD-389/NADPH/PROTON.36.,38938.000000,Old Reactions,False
RXN0-7024,1660.000000,New Reactions,1660.0
UREIDOGLYCOLATE-LYASE-RXN,597522.368421,New Reactions,False
URUR-RXN,638120.368421,Old Reactions,False
TEMP-TRANSPORT-OXAMATE[e]->[p],-0.000000,TEMP,False
TEMP-TRANSPORT-OXAMATE[e]->[p] (reverse),40598.000000,TEMP,False
TEMP-TRANSPORT-OXAMATE[p]->[c],-0.000000,TEMP,False
TEMP-TRANSPORT-OXAMATE[p]->[c] (reverse),40598.000000,TEMP,False
TEMP-OXAMATE-CARBAMOYLTRANSFERASE-RXN (reverse),40598.000000,TEMP,False


## Focus on gene EG11017 --> TREHALA-RXN[CCO-PERI-BAC] (1)

In [428]:
# print TREHALA-RXN[CCO-PERI-BAC] (1) reaction stoichiometry
EG11017_S = S.loc[:, ['TREHALA-RXN[CCO-PERI-BAC]-TREHALOSE/WATER//GLC/ALPHA-GLUCOSE.49.']]
EG11017_S = EG11017_S.loc[~np.all(EG11017_S == 0, axis=1), :]
EG11017_S

,TREHALA-RXN[CCO-PERI-BAC]-TREHALOSE/WATER//GLC/ALPHA-GLUCOSE.49.
ALPHA-GLUCOSE[p],1
GLC[p],1
TREHALOSE[p],-1
WATER[p],-1


No DEM amongst the metabolites in the reaction. However, TREHALOSE is not produced anywhere in the model. There are only transport terms [e] -> [p] -> [c].
Focus on TREHALOSE[p]. EG11017 makes TREHALAPERI-MONOMER. If we allow TREHALOSE uptake, it is almost guaranteed that TREHALA-RXN[CCO-PERI-BAC] will be used. And TREHALOSE is an alternative carbon source for e.coli

In [429]:
uptake_addition = set([
                       'TREHALOSE[e]',
                       ])
uptake_removal = set([
                      # "OXYGEN-MOLECULE[p]",
                      'GLC[p]',
                     ])
new_exchange_molecules = set(['TREHALOSE[e]'])

force_reaction = ['TREHALOSEPHOSPHA-RXN__TREHALOSEPHOSPHASYN-MONOMER']
# force_reaction = None
objective_weights = {'secretion': 0.01, 'efficiency': 0.000001, 'kinetics': 0.001}

In [430]:
solution_flux, test_reaction_names, S_new, test_metabolites, test_kinetic = test_NetworkFlowModel(
                                            objective_weights, 
                                            uptake_addition=uptake_addition, uptake_removal=uptake_removal,
                                            new_exchange_molecules=new_exchange_molecules,
                                            # force_reaction=force_reaction
)    

Force Reaction ID is: None
Optimal objective function reached is: 39540.102149225895


In [431]:
sim_flux = pd.DataFrame({
    'flux': solution_flux,
    'is_new': [
        'New Reactions' if id in fba_new_reaction_ids 
        else 'TEMP' if id in add_reaction.keys() 
        else 'Old Reactions' 
            for id in test_reaction_names
    ]
}, index=test_reaction_names)

In [432]:
met_of_interest = ['TREHALOSE[p]', 'TREHALOSE[c]', 'GLC[p]', 'ALPHA-GLUCOSE[p]', 'TREHALOSE-6P[c]', 'Glucopyranose[c]']
S_new = pd.DataFrame(S_new, index=test_metabolites, columns=test_reaction_names)
S_met, rxns  = get_subset_S(S_new, met_of_interest)

In [433]:
rxn_flux = sim_flux.loc[rxns]
rxn_flux['kinetic'] = [test_kinetic[r] if r in kinetic_reaction_ids else False for r in rxn_flux.index]
rxn_flux

,flux,is_new,kinetic
2.4.1.230-RXN,0.0,Old Reactions,False
2.4.1.230-RXN (reverse),-0.0,Old Reactions,False
3.2.1.21-RXN-Beta-D-glucosides/WATER//Non-Glucosylated-Glucose-Acceptors/ALPHA-GLUCOSE.74.,-0.0,Old Reactions,False
3.2.1.21-RXN-Beta-D-glucosides/WATER//Non-Glucosylated-Glucose-Acceptors/GLC.64.,-0.0,Old Reactions,False
6-PHOSPHO-BETA-GLUCOSIDASE-RXN,-0.0,New Reactions,False
...,...,...,...
TREHALOSE6PSYN-RXN,-0.0,Old Reactions,513.0
TREHALOSE6PSYN-RXN-CPD-12575/ALPHA-GLC-6-P//UDP/TREHALOSE-6P/PROTON.49.,-0.0,Old Reactions,False
TREHALOSE6PSYN-RXN-CPD-12575/GLC-6-P//UDP/TREHALOSE-6P/PROTON.43.,-0.0,Old Reactions,False
TREHALOSEPHOSPHA-RXN,-0.0,Old Reactions,False


What's up with TREHALOSEPHOSPHA-RXN. Not so related, but let's deviate slightly to see if we can figure something out over here.

In [434]:
met_of_interest = ['TREHALOSE-6P[c]', 'TREHALOSE[c]', 'TREHALOSE[p]']
# met_of_interest = ['TREHALOSE-6P[c]']
# met_of_interest = ['GLC-1-P[c]']
met_of_interest = ['CPD-12575[c]']
met_of_interest = ['D-glucopyranose-6-phosphate[c]']
S_new = pd.DataFrame(S_new, index=test_metabolites, columns=test_reaction_names)
S_met, rxns  = get_subset_S(S_new, met_of_interest)
S_met

,6-PHOSPHO-BETA-GLUCOSIDASE-RXN,6-PHOSPHO-BETA-GLUCOSIDASE-RXN-CPD-15978/WATER//D-glucopyranose-6-phosphate/ALPHA-GLUCOSE.59.,6-PHOSPHO-BETA-GLUCOSIDASE-RXN-CPD-15978/WATER//D-glucopyranose-6-phosphate/GLC.49.,6-PHOSPHO-BETA-GLUCOSIDASE-RXN-CPD-507/WATER//D-glucopyranose-6-phosphate/ALPHA-GLUCOSE.57.,6-PHOSPHO-BETA-GLUCOSIDASE-RXN-CPD-507/WATER//D-glucopyranose-6-phosphate/GLC.47.,6-PHOSPHO-BETA-GLUCOSIDASE-RXN-CPD-507/WATER//D-glucopyranose-6-phosphate/Glucopyranose.57.,GLU6PDEHYDROG-RXN,GLUCOKIN-RXN,GLUCOKIN-RXN-ALPHA-GLUCOSE/ATP//D-glucopyranose-6-phosphate/ADP/PROTON.58.,PHOSPHOGLUCMUT-RXN,...,TRANS-RXN-157-Hpr-pi-phospho-L-histidines/GLC//D-glucopyranose-6-phosphate/Hpr-Histidine.75.,TRANS-RXN-157-Hpr-pi-phospho-L-histidines/Glucopyranose//D-glucopyranose-6-phosphate/Hpr-Histidine.85.,TRANS-RXN-33,TRANS-RXN-33-ALPHA-GLC-6-P/Pi//D-glucopyranose-6-phosphate/Pi.49.,TRANS-RXN-33-D-glucopyranose-6-phosphate/Pi//D-glucopyranose-6-phosphate/Pi.63.,TRANS-RXN-33-GLC-6-P/Pi//D-glucopyranose-6-phosphate/Pi.43.,TRE6PHYDRO-RXN,TRE6PHYDRO-RXN-TREHALOSE-6P/WATER//D-glucopyranose-6-phosphate/ALPHA-GLUCOSE.62.,TRE6PHYDRO-RXN-TREHALOSE-6P/WATER//D-glucopyranose-6-phosphate/GLC.52.,TREHALOSE6PSYN-RXN
D-glucopyranose-6-phosphate[c],1,1,1,1,1,1,-1,1,1,1,...,1,1,1,1,1,1,1,1,1,-1


In [435]:
reaction_names.index('TRE6PHYDRO-RXN')
# reaction_names[1565:1575]

9269

In [436]:
rxns_of_interest = ['GLUC1PURIDYLTRANS-RXN', 'TREHALOSEPHOSPHA-RXN__TREHALOSEPHOSPHASYN-MONOMER',
                    'TREHALOSE6PSYN-RXN']
rxn_flux = sim_flux.loc[reaction_names[9269:9279]]
rxn_flux['kinetic'] = [test_kinetic[r] if r in kinetic_reaction_ids else False for r in rxn_flux.index]
rxn_flux


,flux,is_new,kinetic
TRE6PHYDRO-RXN,-0.000000,Old Reactions,False
TRE6PHYDRO-RXN-TREHALOSE-6P/WATER//ALPHA-GLC-6-P/ALPHA-GLUCOSE.48.,-0.000000,Old Reactions,False
TRE6PHYDRO-RXN-TREHALOSE-6P/WATER//ALPHA-GLC-6-P/GLC.38.,-0.000000,Old Reactions,False
TRE6PHYDRO-RXN-TREHALOSE-6P/WATER//ALPHA-GLC-6-P/Glucopyranose.48.,198559.197115,Old Reactions,False
TRE6PHYDRO-RXN-TREHALOSE-6P/WATER//D-glucopyranose-6-phosphate/ALPHA-GLUCOSE.62.,-0.000000,Old Reactions,False
TRE6PHYDRO-RXN-TREHALOSE-6P/WATER//D-glucopyranose-6-phosphate/GLC.52.,-0.000000,Old Reactions,False
TRE6PHYDRO-RXN-TREHALOSE-6P/WATER//GLC-6-P/ALPHA-GLUCOSE.42.,-0.000000,Old Reactions,False
TRE6PHYDRO-RXN-TREHALOSE-6P/WATER//GLC-6-P/GLC.32.,-0.000000,Old Reactions,False
TRE6PHYDRO-RXN-TREHALOSE-6P/WATER//GLC-6-P/Glucopyranose.42.,-0.000000,Old Reactions,False
TREHALA-RXN[CCO-CYTOSOL]-TREHALOSE/WATER//GLC/ALPHA-GLUCOSE.48.,-0.000000,Old Reactions,False


## Focus on gene EG10369 --> RXN0-6373 (3)

In [8]:
rxns_of_interest = ['RXN0-6373-ALPHA-GLUCOSE/UBIQUINONE-8//GLC-D-LACTONE/CPD-9956.51.',
                    'RXN0-6373-GLC/UBIQUINONE-8//GLC-D-LACTONE/CPD-9956.41.',
                    'RXN0-6373-Glucopyranose/UBIQUINONE-8//GLC-D-LACTONE/CPD-9956.51.',]
# print stoichiometry
EG10369_S = S.loc[:, rxns_of_interest]
EG10369_S = EG10369_S.loc[~np.all(EG10369_S == 0, axis=1), :]
EG10369_S

,RXN0-6373-ALPHA-GLUCOSE/UBIQUINONE-8//GLC-D-LACTONE/CPD-9956.51.,RXN0-6373-GLC/UBIQUINONE-8//GLC-D-LACTONE/CPD-9956.41.,RXN0-6373-Glucopyranose/UBIQUINONE-8//GLC-D-LACTONE/CPD-9956.51.
ALPHA-GLUCOSE[p],-1,0,0
CPD-9956[i],1,1,1
GLC-D-LACTONE[p],1,1,1
GLC[p],0,-1,0
Glucopyranose[p],0,0,-1
UBIQUINONE-8[i],-1,-1,-1


In [9]:
rxn_flux = sim_flux.loc[rxns_of_interest]
rxn_flux['kinetic'] = [test_kinetic[r] if r in kinetic_reaction_ids else False for r in rxn_flux.index]
rxn_flux

NameError: name 'sim_flux' is not defined

In [33]:
uptake_addition = set([
                       # 'TREHALOSE[e]',
                       ])
uptake_removal = set([
                      # "OXYGEN-MOLECULE[p]",
                      # 'GLC[p]',
                     ])

# add_reaction = {
#     'TEMP-TRANSPORT-GLC-D-LACTONE[p]->[c]': {
#         'GLC-D-LACTONE[p]': -1,
#         'GLC-D-LACTONE[c]': 1
#     },
#     'TEMP-TRANSPORT-GLC-D-LACTONE[p]->[c](reverse)': {
#         'GLC-D-LACTONE[p]': 1,
#         'GLC-D-LACTONE[c]': -1
#     },
# }

add_reaction = {
    'TEMP-GLUCONOLACT-RXN[p]':{
        "GLC-D-LACTONE[p]": -1,
        "WATER[p]": -1,
        "GLUCONATE[p]": 1,
        "PROTON[p]": 1
        
    }
}
remove_reaction = ['GLUCONOLACT-RXN']

objective_weights = {'secretion': 0.01, 'efficiency': 0.000001, 'kinetics': 0.01}

In [34]:
solution_flux, test_reaction_names, S_new, test_metabolites, test_kinetic = test_NetworkFlowModel(
                                            objective_weights, 
                                            uptake_addition=uptake_addition, uptake_removal=uptake_removal,
                                            add_reaction=add_reaction, remove_reaction=remove_reaction,
)

(6076, 9359)
Force Reaction ID is: None
Optimal objective function reached is: 177468.98455352083


In [35]:
sim_flux = pd.DataFrame({
    'flux': solution_flux,
    'is_new': [
        'New Reactions' if id in fba_new_reaction_ids 
        else 'TEMP' if id in add_reaction.keys() 
        else 'Old Reactions' 
            for id in test_reaction_names
    ]
}, index=test_reaction_names)

In [36]:
met_of_interest = ['GLC-D-LACTONE[p]', "GLUCONATE[p]",'GLUCONATE[c]']
S_new = pd.DataFrame(S_new, index=test_metabolites, columns=test_reaction_names)
S_met, rxns  = get_subset_S(S_new, met_of_interest)

In [37]:
rxn_flux = sim_flux.loc[rxns]
rxn_flux['kinetic'] = [test_kinetic[r] if r in kinetic_reaction_ids else False for r in rxn_flux.index]
rxn_flux

,flux,is_new,kinetic
1.1.1.215-RXN (reverse),-0.000000,Old Reactions,False
GLUCONATE-5-DEHYDROGENASE-RXN-GLUCONATE/NADP//5-DEHYDROGLUCONATE/NADPH/PROTON.48. (reverse),-0.000000,Old Reactions,False
GLUCONOKIN-RXN,844768.238971,Old Reactions,False
RXN-17754,-0.000000,New Reactions,False
RXN0-5185[CCO-CYTOSOL]-CPD-2961/WATER//GLUCONATE/Pi.42.,-0.000000,Old Reactions,1731.0
RXN0-5185[CCO-PERI-BAC]-CPD-2961/WATER//GLUCONATE/Pi.43.,-0.000000,Old Reactions,False
RXN0-6373-ALPHA-GLUCOSE/UBIQUINONE-8//GLC-D-LACTONE/CPD-9956.51.,-0.000000,New Reactions,False
RXN0-6373-GLC/UBIQUINONE-8//GLC-D-LACTONE/CPD-9956.41.,844774.238971,New Reactions,1631703.0
RXN0-6373-Glucopyranose/UBIQUINONE-8//GLC-D-LACTONE/CPD-9956.51.,-0.000000,New Reactions,False
RXN0-7101,-0.000000,Old Reactions,False


In [18]:
met_of_interest = ['GLC-D-LACTONE[p]','GLC[p]', 'UBIQUINONE-8[i]',]
S_new = pd.DataFrame(S_new, index=test_metabolites, columns=test_reaction_names)
S_met, rxns  = get_subset_S(S_new, met_of_interest)
rxn_flux = sim_flux.loc[rxns]
rxn_flux['kinetic'] = [test_kinetic[r] if r in kinetic_reaction_ids else False for r in rxn_flux.index]
rxn_flux

,flux,is_new,kinetic
3.2.1.21-RXN-Beta-D-glucosides/WATER//Non-Glucosylated-Glucose-Acceptors/GLC.64.,-0.0,Old Reactions,False
BETAGALACTOSID-RXN[CCI-PERI-BAC-GN]-Alpha-lactose/WATER//GALACTOSE/GLC.52.,-0.0,New Reactions,False
BETAGALACTOSID-RXN[CCI-PERI-BAC-GN]-CPD-15972/WATER//GALACTOSE/GLC.48.,-0.0,New Reactions,False
DALADEHYDROG-RXN-D-ALANINE/UBIQUINONE-8/WATER//AMMONIUM/PYRUVATE/CPD-9956.57.,0.0,Old Reactions,False
DIHYDROOROTATE-DEHYDROGENASE-RXN-DI-H-OROTATE/UBIQUINONE-8//OROTATE/CPD-9956.44.,27737.0,Old Reactions,False
...,...,...,...
TRANS-RXN-320-GLC/ATP/WATER//GLC/ADP/Pi/PROTON.33.,-0.0,Old Reactions,False
TRANS-RXN-320-GLC/ATP/WATER//Glucopyranose/ADP/Pi/PROTON.43.,-0.0,Old Reactions,False
TREHALA-RXN[CCO-PERI-BAC]-TREHALOSE/WATER//GLC/ALPHA-GLUCOSE.49.,0.0,New Reactions,3980.0
TEMP-TRANSPORT-GLC-D-LACTONE[p]->[c],6.0,TEMP,False


## Look at genes altogehter
1. EG11017 --> TREHALA-RXN[CCO-PERI-BAC] (1)
2. EG10033 --> 'GLUCOSE-1-PHOSPHAT-RXN[P]' (3) and 'RXN0-1001'
3. EG10369 --> RXN0-6373 (3)

By adding GLC-D-LACTONE [p] -> [c] transport and increase kinetic weight on the objective function (for Gene EG10369), we were able to reach target flow for a lot of the kinetic reactions. Interesting, the two zero flow kinetic reactions are each catalyzed by products of gene EG11017 and gene EG10033. 

In [445]:
uptake_addition = set([
                       'TREHALOSE[e]',
                       ])
uptake_removal = set([
                      # "OXYGEN-MOLECULE[p]",
                      'GLC[p]',
                     ])

new_exchange_molecules = set(['TREHALOSE[e]'])

add_reaction = {
    'TEMP-TRANSPORT-GLC-D-LACTONE[p]->[c]': {
        'GLC-D-LACTONE[p]': -1,
        'GLC-D-LACTONE[c]': 1
    },
    'TEMP-TRANSPORT-GLC-D-LACTONE[p]->[c](reverse)': {
        'GLC-D-LACTONE[p]': 1,
        'GLC-D-LACTONE[c]': -1
    },
}

objective_weights = {'secretion': 0.01, 'efficiency': 0.000001, 'kinetics': 0.1}

In [446]:
solution_flux, test_reaction_names, S_new, test_metabolites, test_kinetic = test_NetworkFlowModel(
                                            objective_weights, 
                                            uptake_addition=uptake_addition, uptake_removal=uptake_removal,
                                            add_reaction=add_reaction, new_exchange_molecules=new_exchange_molecules,
)

(6076, 9360)
Force Reaction ID is: None
Optimal objective function reached is: 962904.7685297651


In [447]:
sim_flux = pd.DataFrame({
    'flux': solution_flux,
    'is_new': [
        'New Reactions' if id in fba_new_reaction_ids 
        else 'TEMP' if id in add_reaction.keys() 
        else 'Old Reactions' 
            for id in test_reaction_names
    ]
}, index=test_reaction_names)

In [448]:
met_of_interest = ['GLC-D-LACTONE[p]', 'GLC[p]', 'UBIQUINONE-8[i]',]
S_new = pd.DataFrame(S_new, index=test_metabolites, columns=test_reaction_names)
S_met, rxns  = get_subset_S(S_new, met_of_interest)
rxn_flux = sim_flux.loc[rxns]
rxn_flux['kinetic'] = [test_kinetic[r] if r in kinetic_reaction_ids else False for r in rxn_flux.index]
rxn_flux

,flux,is_new,kinetic
3.2.1.21-RXN-Beta-D-glucosides/WATER//Non-Glucosylated-Glucose-Acceptors/GLC.64.,-0.0,Old Reactions,False
BETAGALACTOSID-RXN[CCI-PERI-BAC-GN]-Alpha-lactose/WATER//GALACTOSE/GLC.52.,0.0,New Reactions,False
BETAGALACTOSID-RXN[CCI-PERI-BAC-GN]-CPD-15972/WATER//GALACTOSE/GLC.48.,-0.0,New Reactions,False
DALADEHYDROG-RXN-D-ALANINE/UBIQUINONE-8/WATER//AMMONIUM/PYRUVATE/CPD-9956.57.,0.0,Old Reactions,False
DIHYDROOROTATE-DEHYDROGENASE-RXN-DI-H-OROTATE/UBIQUINONE-8//OROTATE/CPD-9956.44.,15948.0,Old Reactions,False
...,...,...,...
TRANS-RXN-320-GLC/ATP/WATER//GLC/ADP/Pi/PROTON.33.,-0.0,Old Reactions,False
TRANS-RXN-320-GLC/ATP/WATER//Glucopyranose/ADP/Pi/PROTON.43.,-0.0,Old Reactions,False
TREHALA-RXN[CCO-PERI-BAC]-TREHALOSE/WATER//GLC/ALPHA-GLUCOSE.49.,3980.0,New Reactions,3980.0
TEMP-TRANSPORT-GLC-D-LACTONE[p]->[c],3980.0,TEMP,False


In [449]:
met_of_interest = ['GLC-D-LACTONE[p]', 'GLC-D-LACTONE[c]','GLUCONATE[c]']
S_new = pd.DataFrame(S_new, index=test_metabolites, columns=test_reaction_names)
S_met, rxns  = get_subset_S(S_new, met_of_interest)
rxn_flux = sim_flux.loc[rxns]
rxn_flux['kinetic'] = [test_kinetic[r] if r in kinetic_reaction_ids else False for r in rxn_flux.index]
rxn_flux

,flux,is_new,kinetic
1.1.1.215-RXN (reverse),-0.0,Old Reactions,False
GLUCONATE-5-DEHYDROGENASE-RXN-GLUCONATE/NADP//5-DEHYDROGLUCONATE/NADPH/PROTON.48. (reverse),-0.0,Old Reactions,False
GLUCONOKIN-RXN,5705.0,Old Reactions,False
GLUCONOLACT-RXN,3980.0,Old Reactions,False
RXN-17754,-0.0,New Reactions,False
RXN0-5185[CCO-CYTOSOL]-CPD-2961/WATER//GLUCONATE/Pi.42.,1731.0,Old Reactions,1731.0
RXN0-6373-ALPHA-GLUCOSE/UBIQUINONE-8//GLC-D-LACTONE/CPD-9956.51.,-0.0,New Reactions,False
RXN0-6373-GLC/UBIQUINONE-8//GLC-D-LACTONE/CPD-9956.41.,3980.0,New Reactions,1631703.0
RXN0-6373-Glucopyranose/UBIQUINONE-8//GLC-D-LACTONE/CPD-9956.51.,-0.0,New Reactions,False
RXN0-7101,-0.0,Old Reactions,False


## Focus on gene EG10033 --> 'GLUCOSE-1-PHOSPHAT-RXN[P]' (3) and 'RXN0-1001',

In [450]:
rxns_of_interest = []
for rxn in reaction_names:
    if 'GLUCOSE-1-PHOSPHAT-RXN[CCO-PERI-BAC]' in rxn:
        rxns_of_interest.append(rxn)
    elif 'RXN0-1001' in rxn:
        rxns_of_interest.append(rxn)
rxns_of_interest

['GLUCOSE-1-PHOSPHAT-RXN[CCO-PERI-BAC]-GLC-1-P/WATER//ALPHA-GLUCOSE/Pi.46.',
 'GLUCOSE-1-PHOSPHAT-RXN[CCO-PERI-BAC]-GLC-1-P/WATER//ALPHA-GLUCOSE/Pi.46.__GLUCOSE-1-PHOSPHAT-CPLX',
 'GLUCOSE-1-PHOSPHAT-RXN[CCO-PERI-BAC]-GLC-1-P/WATER//GLC/Pi.36.',
 'GLUCOSE-1-PHOSPHAT-RXN[CCO-PERI-BAC]-GLC-1-P/WATER//GLC/Pi.36.__GLUCOSE-1-PHOSPHAT-CPLX',
 'GLUCOSE-1-PHOSPHAT-RXN[CCO-PERI-BAC]-GLC-1-P/WATER//Glucopyranose/Pi.46.',
 'GLUCOSE-1-PHOSPHAT-RXN[CCO-PERI-BAC]-GLC-1-P/WATER//Glucopyranose/Pi.46.__GLUCOSE-1-PHOSPHAT-CPLX',
 'RXN0-1001']

In [451]:
# print stoichiometry
EG10033_S = S.loc[:, rxns_of_interest]
EG10033_S = EG10033_S.loc[~np.all(EG10033_S == 0, axis=1), :]
EG10033_S

,GLUCOSE-1-PHOSPHAT-RXN[CCO-PERI-BAC]-GLC-1-P/WATER//ALPHA-GLUCOSE/Pi.46.,GLUCOSE-1-PHOSPHAT-RXN[CCO-PERI-BAC]-GLC-1-P/WATER//ALPHA-GLUCOSE/Pi.46.__GLUCOSE-1-PHOSPHAT-CPLX,GLUCOSE-1-PHOSPHAT-RXN[CCO-PERI-BAC]-GLC-1-P/WATER//GLC/Pi.36.,GLUCOSE-1-PHOSPHAT-RXN[CCO-PERI-BAC]-GLC-1-P/WATER//GLC/Pi.36.__GLUCOSE-1-PHOSPHAT-CPLX,GLUCOSE-1-PHOSPHAT-RXN[CCO-PERI-BAC]-GLC-1-P/WATER//Glucopyranose/Pi.46.,GLUCOSE-1-PHOSPHAT-RXN[CCO-PERI-BAC]-GLC-1-P/WATER//Glucopyranose/Pi.46.__GLUCOSE-1-PHOSPHAT-CPLX,RXN0-1001
ALPHA-GLUCOSE[p],1,1,0,0,0,0,0
CPD-534[p],0,0,0,0,0,0,1
GLC-1-P[p],-1,-1,-1,-1,-1,-1,0
GLC[p],0,0,1,1,0,0,0
Glucopyranose[p],0,0,0,0,1,1,0
MI-HEXAKISPHOSPHATE[p],0,0,0,0,0,0,-1
Pi[p],1,1,1,1,1,1,1
WATER[p],-1,-1,-1,-1,-1,-1,-1


In [452]:
rxn_flux = sim_flux.loc[rxns_of_interest]
rxn_flux['kinetic'] = [test_kinetic[r] if r in kinetic_reaction_ids else False for r in rxn_flux.index]
rxn_flux

,flux,is_new,kinetic
GLUCOSE-1-PHOSPHAT-RXN[CCO-PERI-BAC]-GLC-1-P/WATER//ALPHA-GLUCOSE/Pi.46.,-0.0,New Reactions,False
GLUCOSE-1-PHOSPHAT-RXN[CCO-PERI-BAC]-GLC-1-P/WATER//ALPHA-GLUCOSE/Pi.46.__GLUCOSE-1-PHOSPHAT-CPLX,-0.0,New Reactions,5251.0
GLUCOSE-1-PHOSPHAT-RXN[CCO-PERI-BAC]-GLC-1-P/WATER//GLC/Pi.36.,-0.0,New Reactions,False
GLUCOSE-1-PHOSPHAT-RXN[CCO-PERI-BAC]-GLC-1-P/WATER//GLC/Pi.36.__GLUCOSE-1-PHOSPHAT-CPLX,0.0,New Reactions,5251.0
GLUCOSE-1-PHOSPHAT-RXN[CCO-PERI-BAC]-GLC-1-P/WATER//Glucopyranose/Pi.46.,-0.0,New Reactions,False
GLUCOSE-1-PHOSPHAT-RXN[CCO-PERI-BAC]-GLC-1-P/WATER//Glucopyranose/Pi.46.__GLUCOSE-1-PHOSPHAT-CPLX,-0.0,New Reactions,5251.0
RXN0-1001,-0.0,New Reactions,549.0


In [453]:
# check if CPD-534 is in exchange
exchange_molecules = metabolism_basal.exchange_molecules.copy()
for exchange in exchange_molecules:
    if 'CPD-534' in exchange:
        print(f'CPD-534 is an exchange molecule', exchange)

CPD-534 is an exchange molecule CPD-534[p]


In [454]:
uptake_addition = set([
                        'GLC-1-P[e]',
                       ])
uptake_removal = set([
                       'GLC[p]'
                     ])
new_exchange_molecules = set(['GLC-1-P[e]'])

add_reaction = {
}

objective_weights = {'secretion': 0.01, 'efficiency': 0.000001, 'kinetics': 0.1}

In [455]:
solution_flux, test_reaction_names, S_new, test_metabolites, test_kinetic = test_NetworkFlowModel(
                                            objective_weights, 
                                            uptake_addition=uptake_addition, uptake_removal=uptake_removal,
                                            add_reaction=None, new_exchange_molecules=new_exchange_molecules,
)

Force Reaction ID is: None
Optimal objective function reached is: 994513.6923794332


In [456]:
sim_flux = pd.DataFrame({
    'flux': solution_flux,
    'is_new': [
        'New Reactions' if id in fba_new_reaction_ids 
        else 'TEMP' if id in add_reaction.keys() 
        else 'Old Reactions' 
            for id in test_reaction_names
    ]
}, index=test_reaction_names)

In [457]:
# check rxns_of_interest's metabolites flux
met_of_interest = list(EG10033_S.index)
met_of_interest.remove('WATER[p]')
met_of_interest.remove('Pi[p]')
met_of_interest
S_new = pd.DataFrame(S_new, index=test_metabolites, columns=test_reaction_names)
S_met, rxns  = get_subset_S(S_new, met_of_interest)
rxn_flux = sim_flux.loc[rxns]
rxn_flux['kinetic'] = [test_kinetic[r] if r in kinetic_reaction_ids else False for r in rxn_flux.index]
rxn_flux

,flux,is_new,kinetic
3.2.1.21-RXN,-0.0,Old Reactions,False
3.2.1.21-RXN-Beta-D-glucosides/WATER//Non-Glucosylated-Glucose-Acceptors/ALPHA-GLUCOSE.74.,-0.0,Old Reactions,False
3.2.1.21-RXN-Beta-D-glucosides/WATER//Non-Glucosylated-Glucose-Acceptors/GLC.64.,-0.0,Old Reactions,False
BETAGALACTOSID-RXN[CCI-PERI-BAC-GN]-Alpha-lactose/WATER//GALACTOSE/ALPHA-GLUCOSE.62.,-0.0,New Reactions,False
BETAGALACTOSID-RXN[CCI-PERI-BAC-GN]-Alpha-lactose/WATER//GALACTOSE/GLC.52.,-0.0,New Reactions,False
...,...,...,...
TRANS-RXN0-614-CPD-534//CPD-534.17.,-0.0,Old Reactions,False
TRANS-RXN0-614-CPD-534//CPD-534.17. (reverse),-0.0,Old Reactions,False
TRANS-RXN0-614-MI-HEXAKISPHOSPHATE//MI-HEXAKISPHOSPHATE.41.,-0.0,Old Reactions,False
TRANS-RXN0-614-MI-HEXAKISPHOSPHATE//MI-HEXAKISPHOSPHATE.41. (reverse),-0.0,Old Reactions,False


In [458]:
rxn_flux = sim_flux.loc[rxns_of_interest]
rxn_flux['kinetic'] = [test_kinetic[r] if r in kinetic_reaction_ids else False for r in rxn_flux.index]
rxn_flux

,flux,is_new,kinetic
GLUCOSE-1-PHOSPHAT-RXN[CCO-PERI-BAC]-GLC-1-P/WATER//ALPHA-GLUCOSE/Pi.46.,3.188073e+06,New Reactions,False
GLUCOSE-1-PHOSPHAT-RXN[CCO-PERI-BAC]-GLC-1-P/WATER//ALPHA-GLUCOSE/Pi.46.__GLUCOSE-1-PHOSPHAT-CPLX,5.251000e+03,New Reactions,5251.0
GLUCOSE-1-PHOSPHAT-RXN[CCO-PERI-BAC]-GLC-1-P/WATER//GLC/Pi.36.,-0.000000e+00,New Reactions,False
GLUCOSE-1-PHOSPHAT-RXN[CCO-PERI-BAC]-GLC-1-P/WATER//GLC/Pi.36.__GLUCOSE-1-PHOSPHAT-CPLX,5.251000e+03,New Reactions,5251.0
GLUCOSE-1-PHOSPHAT-RXN[CCO-PERI-BAC]-GLC-1-P/WATER//Glucopyranose/Pi.46.,-0.000000e+00,New Reactions,False
GLUCOSE-1-PHOSPHAT-RXN[CCO-PERI-BAC]-GLC-1-P/WATER//Glucopyranose/Pi.46.__GLUCOSE-1-PHOSPHAT-CPLX,5.251000e+03,New Reactions,5251.0
RXN0-1001,-0.000000e+00,New Reactions,549.0


### Isolate out RXN0-1001 and figure out its pathway

In [459]:
temp = S.loc[:, 'RXN0-1001']
temp = temp.loc[temp!=0]
temp

CPD-534[p]                1
MI-HEXAKISPHOSPHATE[p]   -1
Pi[p]                     1
WATER[p]                 -1
Name: RXN0-1001, dtype: int8

In [460]:
met_of_interest = list(temp.index)
met_of_interest.remove('Pi[p]')
met_of_interest.remove('WATER[p]')

S_new = pd.DataFrame(S_new, index=test_metabolites, columns=test_reaction_names)
S_met, rxns  = get_subset_S(S_new, met_of_interest)
temp = S.loc[:, rxns]
temp = temp.loc[~np.all(temp == 0, axis=1), :]
temp

,RXN-10940,RXN0-1001,TRANS-RXN0-614-CPD-534//CPD-534.17.,TRANS-RXN0-614-CPD-534//CPD-534.17. (reverse),TRANS-RXN0-614-MI-HEXAKISPHOSPHATE//MI-HEXAKISPHOSPHATE.41.,TRANS-RXN0-614-MI-HEXAKISPHOSPHATE//MI-HEXAKISPHOSPHATE.41. (reverse)
CPD-534[e],0,0,1,-1,0,0
CPD-534[p],0,1,-1,1,0,0
MI-HEXAKISPHOSPHATE[e],0,0,0,0,1,-1
MI-HEXAKISPHOSPHATE[p],-1,-1,0,0,-1,1
MI-PENTAKISPHOSPHATE[p],1,0,0,0,0,0
Pi[p],1,1,0,0,0,0
WATER[p],-1,-1,0,0,0,0


In [461]:
rxn_flux = sim_flux.loc[rxns]
rxn_flux['kinetic'] = [test_kinetic[r] if r in kinetic_reaction_ids else False for r in rxn_flux.index]
rxn_flux

,flux,is_new,kinetic
RXN-10940,-0.0,Old Reactions,False
RXN0-1001,-0.0,New Reactions,549.0
TRANS-RXN0-614-CPD-534//CPD-534.17.,-0.0,Old Reactions,False
TRANS-RXN0-614-CPD-534//CPD-534.17. (reverse),-0.0,Old Reactions,False
TRANS-RXN0-614-MI-HEXAKISPHOSPHATE//MI-HEXAKISPHOSPHATE.41.,-0.0,Old Reactions,False
TRANS-RXN0-614-MI-HEXAKISPHOSPHATE//MI-HEXAKISPHOSPHATE.41. (reverse),-0.0,Old Reactions,False
